In [9]:
import qiskit

In [10]:
# simulation of Shrodinger equation for infinite square well potential in 1D on a Quantum Computer
# using Qiskit

import numpy as np
import matplotlib.pyplot as plt
#from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error
from qiskit.providers.aer.noise.errors.standard_errors import thermal_relaxation_error
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel

In [11]:
# Now we will define the parameters of the simulation
# Number of qubits
n = 4
# Number of time steps
time_steps = 100
# Time step
dt = 0.01
# Number of Trotter steps
trotter_steps = 1
# Number of shots
shots = 1000

In [12]:
# Now we will define the potential energy function
def potential(x):
    if x < 0 or x > 1:
        return 1e10
    else:
        return 0
    
# Now we will define the kinetic energy function
def kinetic(p):
    return p**2

In [13]:
# Now we will define the initial state
def initial_state(x):
    if x < 0.5:
        return 1
    else:
        return 0

In [14]:
# Now we will define the quantum circuit
def quantum_circuit(x, p):
    qc = qiskit.QuantumCircuit(n)
    qc.h(0)
    qc.h(1)
    qc.h(2)
    qc.h(3)
    qc.barrier()
    for i in range(n):
        qc.rz(x*dt*p/(2*trotter_steps), i)
    for i in range(n):
        qc.rx(x*dt/(2*trotter_steps), i)
    for i in range(n):
        qc.rz(x*dt*p/(2*trotter_steps), i)
    qc.barrier()
    for i in range(n):
        qc.h(i)
    qc.barrier()
    return qc

# Now we will define the quantum circuit for the potential energy
def quantum_circuit_potential(x):
    qc = qiskit.QuantumCircuit(n)
    qc.h(0)
    qc.h(1)
    qc.h(2)
    qc.h(3)
    qc.barrier()
    for i in range(n):
        qc.rz(x*dt/(2*trotter_steps), i)
    for i in range(n):
        qc.rx(x*dt/(2*trotter_steps), i)
    for i in range(n):
        qc.rz(x*dt/(2*trotter_steps), i)
    qc.barrier()
    for i in range(n):
        qc.h(i)
    qc.barrier()
    return qc

# Now we will define the quantum circuit for the kinetic energy
def quantum_circuit_kinetic(p):
    qc = qiskit.QuantumCircuit(n)
    qc.h(0)
    qc.h(1)
    qc.h(2)
    qc.h(3)
    qc.barrier()
    for i in range(n):
        qc.rz(p*dt/(2*trotter_steps), i)
    for i in range(n):
        qc.rx(p*dt/(2*trotter_steps), i)
    for i in range(n):
        qc.rz(p*dt/(2*trotter_steps), i)
    qc.barrier()
    for i in range(n):
        qc.h(i)
    qc.barrier()
    return qc


In [15]:
# Now we will define the quantum circuit for the time evolution
def quantum_circuit_time_evolution(x, p):
    qc = quantum_circuit(x, p)
    qc = qc + quantum_circuit_potential(x)
    qc = qc + quantum_circuit_kinetic(p)
    return qc



In [17]:
#Now simulate the quantum circuit
# Initialize the quantum circuit
qc = qiskit.QuantumCircuit(n)

# Initialize the quantum simulator
simulator = AerSimulator()

# Initialize the noise model
noise_model = NoiseModel()

# Define depolarizing error
error = depolarizing_error(0.01, 1)

# Add depolarizing error to noise model
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])

# Define thermal relaxation error
T1 = 100
T2 = 120
error = thermal_relaxation_error(T1, T2, dt)

# Add thermal relaxation error to noise model
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])


c:\Users\Lumin\anaconda3\envs\qc\lib\site-packages\qiskit\providers\aer\noise\errors\quantum_error.py:544: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  choi_sum = choi_sum + prob * iden.compose(Kraus(instr['params']),


In [18]:
# now we will simulate the quantum circuit
for x in range(time_steps):
    for p in range(time_steps):
        qc = quantum_circuit_time_evolution(x, p)
        result = qiskit.execute(qc, simulator, noise_model=noise_model, shots=shots).result()
        counts = result.get_counts(qc)
        print(counts)
        plt.plot(counts)
        plt.show()
        plot_histogram(counts)
        plt.show()
        
        

C:\Users\Lumin\AppData\Local\Temp\ipykernel_17956\2635344116.py:4: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc = qc + quantum_circuit_potential(x)
c:\Users\Lumin\anaconda3\envs\qc\lib\site-packages\qiskit\circuit\quantumcircuit.py:933: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)


QiskitError: 'No counts for experiment "<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000022D3B9CEE20>"'